This file will extract the left images and right images from a rosbag file, and save the images as .png pictures. 

Thie file will create folders to save left and right images.

The .png files will be named by frame (timestamp) numbers.

In [1]:
import rosbag2_py
import cv_bridge
import os
import cv2
from sensor_msgs.msg import Image
from rclpy.serialization import deserialize_message
from rosidl_runtime_py.utilities import get_message

In [2]:
# Bag File Path, change if you want to, 
# this one is just for testing
bag_file_path = '/home/yz27246-admin/test_bag'
# Image Topics, just left right images from ONE camera
CAM1_L = '/cam_1/zed_node/left/image_rect_color'
CAM1_R = '/cam_1/zed_node/right/image_rect_color'

In [3]:
reader = rosbag2_py.SequentialReader()
storage_options = rosbag2_py.StorageOptions(uri=bag_file_path)
converter_options = rosbag2_py.ConverterOptions(
        input_serialization_format='cdr', output_serialization_format='cdr')
reader.open(storage_options, converter_options)

[INFO] [1707260106.150250366] [rosbag2_storage]: Opened database '/home/yz27246-admin/test_bag/test_bag_0.db3' for READ_ONLY.


In [4]:
bridge = cv_bridge.CvBridge()

In [5]:
# Create directories for saving images
import os
left_images_dir = 'left_images'
right_images_dir = 'right_images'
os.makedirs(left_images_dir, exist_ok=True)
os.makedirs(right_images_dir, exist_ok=True)

In [6]:
msg_type = get_message('sensor_msgs/msg/Image')

In [7]:
while reader.has_next():
    topic_name, data, timestamp = reader.read_next()
    if topic_name == CAM1_L:
        image_msg = deserialize_message(data, msg_type)
        cv_image = bridge.imgmsg_to_cv2(image_msg, desired_encoding='bgr8')
        image_filename = f"{left_images_dir}/frame_{timestamp}.png"
        cv2.imwrite(image_filename, cv_image)

    if topic_name == CAM1_R:
        image_msg = deserialize_message(data, msg_type)
        cv_image = bridge.imgmsg_to_cv2(image_msg, desired_encoding='bgr8')
        image_filename = f"{right_images_dir}/frame_{timestamp}.png"
        cv2.imwrite(image_filename, cv_image)